In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.preprocessing import StandardScaler,LabelEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.metrics import f1_score

In [2]:
#reading the train dataset
df_train = pd.read_csv("/content/traindata.csv")
df_train

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,tcp,http,SF,295,2584,0,0,0,...,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21,0
1,1,0,tcp,http,SF,312,839,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21,0
2,2,0,tcp,ftp,SF,1225,2451,0,0,0,...,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11,1
3,3,0,tcp,http,SF,293,296,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21,0
4,4,0,tcp,X11,REJ,0,0,0,0,0,...,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22997,22997,0,tcp,http,REJ,0,0,0,0,0,...,1.00,0.00,0.25,0.15,0.0,0.0,1.00,0.87,21,0
22998,22998,0,tcp,telnet,S0,0,0,0,0,0,...,0.05,0.05,0.00,0.00,1.0,1.0,0.00,0.00,19,1
22999,22999,0,tcp,private,S0,0,0,0,0,0,...,0.06,0.08,0.00,0.00,1.0,1.0,0.00,0.00,21,1
23000,23000,0,udp,other,SF,516,4,0,0,0,...,1.00,0.00,1.00,0.00,0.0,0.0,0.00,0.00,17,0


In [3]:
#reading the test dataset
df_test = pd.read_csv("/content/testdata.csv")
df_test

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level
0,0,4,tcp,pop_3,SF,30,93,0,0,0,...,232,0.91,0.04,0.00,0.00,0.0,0.0,0.0,0.0,16
1,1,0,tcp,private,S0,0,0,0,0,0,...,5,0.02,0.08,0.00,0.00,1.0,1.0,0.0,0.0,21
2,2,0,udp,private,SF,52,54,0,0,0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,17
3,3,0,tcp,http,SF,367,588,0,0,0,...,255,1.00,0.00,0.12,0.06,0.0,0.0,0.0,0.0,21
4,4,0,icmp,ecr_i,SF,1008,0,0,0,0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22496,22496,0,tcp,http,SF,298,2248,0,0,0,...,255,1.00,0.00,0.01,0.01,0.0,0.0,0.0,0.0,21
22497,22497,281,tcp,ftp,SF,160,599,0,0,0,...,55,0.34,0.04,0.01,0.00,0.0,0.0,0.0,0.0,11
22498,22498,0,tcp,private,S0,0,0,0,0,0,...,18,0.07,0.06,0.00,0.00,1.0,1.0,0.0,0.0,21
22499,22499,0,tcp,http,SF,199,11107,0,0,0,...,255,1.00,0.00,0.06,0.04,0.0,0.0,0.0,0.0,21


In [4]:
#shape of datasets
print(df_train.shape)
print(df_test.shape)

(23002, 44)
(22501, 43)


In [5]:
print("Missing values in training set:")
print(df_train.isnull().sum().sum())

print("Missing values in test set:")
print(df_test.isnull().sum().sum())

Missing values in training set:
0
Missing values in test set:
0


In [6]:
df_train.drop('level', axis=1, inplace=True)
df_test.drop('level', axis=1, inplace=True)

In [7]:
#datatypes of columns in datasets
print(df_train.dtypes)

ID                               int64
duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate              

In [8]:
#datatypes of columns in datasets
print(df_test.dtypes)

ID                               int64
duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate              

In [9]:
#encodeing the categorical data to numerical
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_train[["protocol_type", "service","flag"]] = encoder.fit_transform(df_train[["protocol_type", "service", "flag"]])
df_test[["protocol_type", "service", "flag"]] = encoder.transform(df_test[["protocol_type", "service", "flag"]])

In [10]:
#checking imbalanced data
df_train['Class'].value_counts()

,count
Class,
0,12001
1,11001


In [11]:
#splitting the train data
y_train = df_train['Class']
X_train = df_train.drop('Class', axis=1)

In [12]:
# Scale numerical features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(df_test)

In [13]:
#feature selection
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

selector = SelectFromModel(rf, threshold="median", prefit=True)

X_train_selected = selector.transform(X_train)
df_test_selected = selector.transform(df_test)

print("Original Features:", X_train.shape[1])
print("Selected Features:", X_train_selected.shape[1])

Original Features: 42
Selected Features: 21


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [14]:
tree = DecisionTreeClassifier(
    random_state=0,
    ccp_alpha=0.009,
    max_depth=8,
    min_samples_split=10,
    min_samples_leaf=5
    )
tree.fit(X_train_selected, y_train)

DecisionTreeClassifier(ccp_alpha=0.009406156206586185, max_depth=8,
                       min_samples_leaf=5, min_samples_split=10,
                       random_state=0)

In [15]:
# Get predictions
y_train_pred = tree.predict(X_train_selected)
y_test_pred = tree.predict(df_test_selected)

train_f1 = f1_score(y_train, y_train_pred)

print("F1 Score on Training Set: {:.3f}".format(train_f1))

F1 Score on Training Set: 0.960


In [17]:
# Create a DataFrame with ID and Predictions
df_submission = pd.DataFrame({
    'ID': df_test['ID'],
    'Class': y_test_pred
})

# Save as CSV without index
df_submission.to_csv("submission.csv", index=False)

print("Predictions saved to submission.csv")


Predictions saved to submission.csv
